In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import metpy.calc as mpcalc
from metpy.interpolate import cross_section
from metpy.units import units
from herbie import Herbie

# Define the start and end points for our cross section
# Heber City, UT: 40.5074° N, 111.4096° W
# Dinosaur, CO: 40.2436° N, 109.0173° W
start = (40.5074, -111.4096)  # (lat, lon) for Heber City, UT
end = (40.2436, -109.0173)    # (lat, lon) for Dinosaur, CO

# Set up the time and forecast information
model_run = "2025-01-31 12:00"
valid_time = "2025-01-31 18:00"
fxx = 6  # 6-hour forecast from 12Z to get 18Z


In [2]:
# Create a Herbie object for the GFS model
H = Herbie(
    model_run,
    model="gfs",
    product="pgrb2.0p25",  # 0.25-degree resolution GFS data
    fxx=fxx
)

pattern = "TMP:.*mb|HGT:.*mb|HGT:surface"
local_file = H.download(pattern)
ds = H.xarray(pattern, decode_timedelta=False)
ds

✅ Found ┊ model=gfs ┊ product=pgrb2.0p25 ┊ 2025-Jan-31 12:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws


/Users/johnlawson/anaconda3/envs/brc-tools-polars/lib/python3.12/site-packages/herbie/core.py:1106: UserWarning: Will not remove GRIB file because it previously existed.
  warnings.warn("Will not remove GRIB file because it previously existed.")
/Users/johnlawson/anaconda3/envs/brc-tools-polars/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/Users/johnlawson/anaconda3/envs/brc-tools-polars/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/Users/johnl

Note: Returning a list of [4] xarray.Datasets because cfgrib opened with multiple hypercubes.


/Users/johnlawson/anaconda3/envs/brc-tools-polars/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/Users/johnlawson/anaconda3/envs/brc-tools-polars/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(
/Users/johnlawson/anaconda3/envs/brc-tools-polars/lib/python3.12/site-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to

[<xarray.Dataset> Size: 66MB
 Dimensions:              (isobaricInPa: 8, latitude: 721, longitude: 1440)
 Coordinates:
     time                 datetime64[ns] 8B 2025-01-31T12:00:00
     step                 timedelta64[ns] 8B 06:00:00
   * isobaricInPa         (isobaricInPa) float64 64B 70.0 40.0 20.0 ... 2.0 1.0
   * latitude             (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
   * longitude            (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
     valid_time           datetime64[ns] 8B 2025-01-31T18:00:00
     gribfile_projection  object 8B None
 Data variables:
     t                    (isobaricInPa, latitude, longitude) float32 33MB ...
     gh                   (isobaricInPa, latitude, longitude) float32 33MB ...
 Attributes:
     GRIB_edition:            2
     GRIB_centre:             kwbc
     GRIB_centreDescription:  US National Weather Service - NCEP
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:          

In [3]:
# Create a cross-section object
data_crs = ds.metpy.cartopy_crs

# Extract the pressure levels
pressure_levels = ds['isobaric'].metpy.convert_units('hPa')

# Create the cross-section
cross = cross_section(ds, start, end, steps=100)

# Calculate potential temperature
cross['potential_temperature'] = mpcalc.potential_temperature(
    cross['isobaric'],
    cross['TMP']
)

# Extract terrain height along the cross-section
terrain = cross['HGT_surface']


AttributeError: 'list' object has no attribute 'metpy'

In [ ]:
# Create the figure and primary axes
fig = plt.figure(figsize=(15, 10))
ax = plt.axes()

# Plot potential temperature using contour
levels = np.arange(260, 400, 5)
theta_contour = ax.contour(
    cross['lon'],
    cross['isobaric'],
    cross['potential_temperature'],
    levels=levels,
    colors='k',
    linewidths=1.5
)

# Add contour labels
plt.clabel(theta_contour, fontsize=10, inline=1, inline_spacing=10, fmt='%d')

# Plot terrain profile at the bottom
# Convert terrain height to pressure using a simple approximation
# This is a simplification - in practice, you might want a more accurate representation
terrain_pressure = 1013.25 * np.exp(-terrain / 7000)  # Simple pressure-height relationship
ax.fill_between(cross['lon'], 1050, terrain_pressure, color='saddlebrown', alpha=0.8)

# Adjust the y-axis to be logarithmic
ax.set_yscale('symlog')
ax.set_ylim(1050, 100)  # From surface to about 100 hPa
ax.set_yticks(np.arange(1000, 100, -100))
ax.set_yticklabels([f"{p}" for p in np.arange(1000, 100, -100)])
ax.invert_yaxis()  # Invert to have surface at bottom

# Add a map inset showing the cross-section path
ax_inset = fig.add_axes([0.15, 0.15, 0.2, 0.2], projection=data_crs)
ax_inset.coastlines()
ax_inset.add_feature(cfeature.STATES.with_scale('50m'), edgecolor='k', alpha=0.3)

# Plot the path of the cross section
endpoints = data_crs.transform_points(ccrs.Geodetic(),
                                      *np.vstack([start, end]).transpose()[::-1])
ax_inset.scatter(endpoints[:, 0], endpoints[:, 1], c='r', zorder=2)
ax_inset.plot([cross['x'].values[0], cross['x'].values[-1]],
              [cross['y'].values[0], cross['y'].values[-1]], c='r', zorder=2)

# Set map boundaries to focus on our area of interest
ax_inset.set_extent([start[1]-3, end[1]+3, start[0]-3, end[0]+3], crs=ccrs.PlateCarree())

# Set titles and labels
ax.set_title(f'GFS Cross-Section: Heber City, UT to Dinosaur, CO\n'
             f'Valid: {valid_time} (F{fxx:02d} from {model_run})\n'
             f'Potential Temperature (K) and Terrain Profile', fontsize=14)
ax.set_xlabel('Longitude (°E)', fontsize=12)
ax.set_ylabel('Pressure (hPa)', fontsize=12)

# Add a text box with location information
plt.figtext(0.15, 0.9, f'Heber City, UT: {start[0]:.2f}°N, {start[1]:.2f}°E\n'
                       f'Dinosaur, CO: {end[0]:.2f}°N, {end[1]:.2f}°E',
            bbox=dict(facecolor='white', alpha=0.7))

plt.tight_layout()
plt.show()
